In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [5]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.109908,False,25,1,38.0
1,7.588311,False,19,1,40.0
2,4.812691,True,35,1,33.0
3,7.251004,True,30,2,38.0
4,6.206013,False,21,1,36.0


In [6]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9994.000000,10000.000000,10000.000000,9873.000000
mean,7.267343,27.299200,1.035300,38.662919
std,1.345904,6.190841,0.197124,2.560298
min,0.595248,13.000000,1.000000,19.000000
25%,6.563162,22.000000,1.000000,38.000000
50%,7.373360,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,13.000660,54.000000,4.000000,47.000000


In [7]:
df['is_male'].value_counts()

True     5097
False    4903
Name: is_male, dtype: int64

In [8]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [9]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [10]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [12]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [13]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)


Train on 6661 samples, validate on 741 samples
Epoch 1/10
6661/6661 [==============================] - 2s 318us/sample - loss: 3.4772 - mae: 1.1769 - mse: 3.4772 - val_loss: 1.8911 - val_mae: 1.1219 - val_mse: 1.8911
Epoch 2/10
6661/6661 [==============================] - 1s 124us/sample - loss: 1.4309 - mae: 0.9334 - mse: 1.4309 - val_loss: 1.3176 - val_mae: 0.9102 - val_mse: 1.3176
Epoch 3/10
6661/6661 [==============================] - 1s 136us/sample - loss: 1.4196 - mae: 0.9307 - mse: 1.4196 - val_loss: 1.3215 - val_mae: 0.8961 - val_mse: 1.3215
Epoch 4/10
6661/6661 [==============================] - 1s 163us/sample - loss: 1.3621 - mae: 0.9114 - mse: 1.3621 - val_loss: 1.1537 - val_mae: 0.8366 - val_mse: 1.1537
Epoch 5/10
6661/6661 [==============================] - 1s 113us/sample - loss: 1.3470 - mae: 0.9048 - mse: 1.3470 - val_loss: 2.5951 - val_mae: 1.3664 - val_mse: 2.5951
Epoch 6/10
6661/6661 [==============================] - 1s 113us/sample - loss: 1.3224 - mae: 0.8970 - 

In [14]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [16]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.316289
Actual val:  5.37486994756

Predicted val:  7.997316
Actual val:  10.00016820432

Predicted val:  7.9241433
Actual val:  8.75014717878

Predicted val:  7.352114
Actual val:  6.4374980503999994

Predicted val:  8.0149355
Actual val:  9.25059651352

Predicted val:  6.5404973
Actual val:  4.2218523173

Predicted val:  7.790035
Actual val:  8.5208664263

Predicted val:  8.385647
Actual val:  8.0248263368

Predicted val:  7.316289
Actual val:  6.8122838958

Predicted val:  8.000883
Actual val:  7.403122757959999



In [17]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [18]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [19]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…